## Spring boot met docker

In deze zelfstudie zullen we ons concentreren op het dockerize van een _Spring Boot Application_ om deze uit te voeren in een geïsoleerde omgeving, ook wel _container_ genoemd.

We zullen leren hoe we een samenstelling van containers kunnen maken, die van elkaar afhankelijk zijn en tegen elkaar zijn gekoppeld in een virtueel privénetwerk. We zullen ook zien hoe ze samen met enkele opdrachten kunnen worden beheerd.

Laten we beginnen met het maken van een eenvoudige Spring Boot-applicatie die we vervolgens uitvoeren in een lichtgewicht basisimage met [_Alpine Linux_](https://hub.docker.com/_/alpine/).

Als voorbeeld van een toepassing die we kunnen dockeriseren, maken we een eenvoudige Spring _Boot-applicatie, docker-message-server,_ die één eindpunt blootlegt en een statisch bericht retourneert:

```
@RestController
public class DockerMessageController {
    @GetMapping("/messages")
    public String getMessage() {
        return "Hello from Docker!";
    }
}
```

Met een correct geconfigureerd Maven-bestand kunnen we vervolgens een uitvoerbaar jar-bestand maken:

```
$> mvn clean package
```

Vervolgens starten we de Spring Boot-applicatie op:

```
$> java -jar target/docker-message-server-1.0.0.jar
```

Nu hebben we een werkende Spring Boot-applicatie die we kunnen openen op _localhost: 8888 / berichten._

Om de toepassing te dockeriseren, maken we eerst een bestand met de naam _Dockerfile_ met de volgende inhoud:

```
FROM openjdk:8-jdk-alpine
MAINTAINER baeldung.com
COPY target/docker-message-server-1.0.0.jar message-server-1.0.0.jar
ENTRYPOINT ["java","-jar","/message-server-1.0.0.jar"]
```

Dit bestand bevat de volgende informatie:

-   **VAN**: Als basis voor onze afbeelding nemen we de _Java-compatibele_ _Alpine Linux_ die in de vorige sectie is gemaakt.
-   **ONDERHOUDER**: De onderhouder van het beeld.
-   **KOPIËREN**: We laten _Docker_ ons potbestand naar de afbeelding kopiëren.
-   **ENTRYPOINT**: Dit is het uitvoerbare bestand dat moet worden gestart wanneer de container opstart. We moeten ze definiëren als _JSON-Array_ omdat we een _ENTRYPOINT_ in combinatie met een _CMD_ gebruiken voor sommige toepassingsargumenten.

Om een image te maken van onze _Dockerfile_, moeten we _'docker build'_ uitvoeren zoals voorheen:

[![freestar](https://a.pub.network/core/imgs/fslogo-green.svg)](https://freestar.com/?utm_campaign=branding&utm_medium=&utm_source=baeldung.com&utm_content=baeldung_leaderboard_mid_2)

AD

```
$> docker build --tag=message-server:latest .
```

Ten slotte kunnen we de container uitvoeren vanuit onze afbeelding:

```
$> docker run -p8887:8888 message-server:latest
```

This will start our application in Docker, and we can access it from the host machine at _localhost:8887/messages_. Here it's important to define the port mapping, which maps a port on the host (8887) to the port inside Docker (_8888_). This is the port we defined in the properties of the Spring Boot application.

Note: Port 8887 might not be available on the machine where we launch the container. In this case, the mapping might not work and we need to choose a port that's still available.

If we run the container in detached mode, we can inspect its details, stop it, and remove it with the following commands:

```
$> docker inspect message-server
$> docker stop message-server
$> docker rm message-server
```

### 2.1. Changing the Base Image[](https://www.baeldung.com/dockerizing-spring-boot-application#1-changing-the-base-image)

We can easily change the base image in order to use a different Java version. For example, if we want to use the Corretto distribution from Amazon, we can simply change the _Dockerfile_:

```
FROM amazoncorretto:11-alpine-jdk
MAINTAINER baeldung.com
COPY target/docker-message-server-1.0.0.jar message-server-1.0.0.jar
ENTRYPOINT ["java","-jar","/message-server-1.0.0.jar"]
```

Furthermore, we can use a custom base image. We'll look at how to do that later in this tutorial.

## **3\. Dockerize Applications in a Composite**[](https://www.baeldung.com/dockerizing-spring-boot-application#Composite)

_Docker_ commands and _Dockerfiles_ are particularly suitable for creating individual containers. However, if we want to _operate on a network of isolated applications_, the container management quickly becomes cluttered.

**To solve this, _Docker_ provides a tool named _Docker Compose_.** This tool comes with its own build-file in _YAML_ format, and is better suited for managing multiple containers. For instance, it's able to start or stop a composite of services in one command, or merges the logging output of multiple services together into one _pseudo-tty_.

### 3.1. The Second Spring Boot Application[](https://www.baeldung.com/dockerizing-spring-boot-application#1-the-second-spring-boot-application)

Let's build an example of two applications running in different Docker containers. They will communicate with each other, and be presented as a “single unit” to the host system. As a simple example, we'll create a second Spring Boot application _docker-product-server_:

```
@RestController
public class DockerProductController {
    @GetMapping("/products")
    public String getMessage() {
        return "A brand new product";
    }
}
```

We can build and start the application in the same way as our _message-server_.

### 3.2. The Docker Compose File[](https://www.baeldung.com/dockerizing-spring-boot-application#2-the-docker-compose-file)

We can combine the configuration for both services in one file called _docker-compose.yml_:

```
version: '2'
services:
    message-server:
        container_name: message-server
        build:
            context: docker-message-server
            dockerfile: Dockerfile
        image: message-server:latest
        ports:
            - 18888:8888
        networks:
            - spring-cloud-network
    product-server:
        container_name: product-server
        build:
            context: docker-product-server
            dockerfile: Dockerfile
        image: product-server:latest
        ports:
            - 19999:9999
        networks:
            - spring-cloud-network
networks:
    spring-cloud-network:
        driver: bridge
```

-   **version**: Specifies which format version should be used. This is a mandatory field. Here we use the newer version, whereas the _legacy format_ is ‘1'.
-   **services**: Each object in this key defines a _service_, a.k.a container. This section is mandatory.
    -   **build**: If given, _docker-compose_ is able to build an image from a _Dockerfile_
        -   **context**: If given, it specifies the build-directory, where the _Dockerfile_ is looked-up.
        -   **dockerfile**: If given, it sets an alternate name for a _Dockerfile._
    -   **image**: Tells _Docker_ which name it should give to the image when build-features are used. Otherwise, it's searching for this image in the library or _remote-registry._
    -   **networks**: This is the identifier of the named networks to use. A given _name-value_ must be listed in the _networks_ section.
-   **networks**: In this section, we're specifying the _networks_ available to our services_._ In this example, we let _docker-compose_ create a named _network_ of type _‘bridge'_ for us. If the option _external_ is set to _true_, it will use an existing one with the given name.

Before we continue, we'll check our build-file for syntax-errors:

```
$> docker-compose config
```

Then we can build our images, create the defined containers, and start it in one command:

```
$> docker-compose up --build
```

This will start up the _message-server_ and _product-server_ in one go.

To stop the containers, remove them from _Docker_ and remove the connected _networks_ from it. To do this, we can use the opposite command:

```
$> docker-compose down
```

For a more detailed introduction to _docker-compose,_ we can read our article [Introduction to Docker Compose](https://www.baeldung.com/docker-compose).

### 3.3. Scaling Services[](https://www.baeldung.com/dockerizing-spring-boot-application#3-scaling-services)

A nice feature of _docker-compose_ is the **ability to scale services**. For example, we can tell _Docker_ to run three containers for the _message-server_ and two containers for the _product-server_.

However, for this to work properly, we have to remove the _container\_name_ from our _docker-compose.yml_, so _Docker_ can choose names, and change the _exposed port configuration_ to avoid clashes.

For the ports, we can tell Docker to map a range of ports on the host to one specific port inside Docker:

```
ports:
    - 18800-18888:8888
```

After that, we're able to scale our services like so (note that we're using a modified _yml-file_):

```
$> docker-compose --file docker-compose-scale.yml up -d --build --scale message-server=1 product-server=1
```

This command will spin up a single _message-server_ and a single _product-server_.

[![freestar](https://a.pub.network/core/imgs/fslogo-green.svg)](https://freestar.com/?utm_campaign=branding&utm_medium=banner&utm_source=baeldung.com&utm_content=baeldung_incontent_2)

AD

To scale our services, we can run the following command:

```
$> docker-compose --file docker-compose-scale.yml up -d --build --scale message-server=3 product-server=2
```

This command will launch **two** additional message servers and **one** additional product server. The running containers won't be stopped.

## **4\. Custom Base Image**[](https://www.baeldung.com/dockerizing-spring-boot-application#custom-base-image)

The base image (_openjdk:8-jdk-alpine_) we have used so far contained a distribution of the Alpine operating system with a JDK 8 already installed. Alternatively, we can build our own base image (based on Alpine or any other operating system).

To do so, we can use a _Dockerfile_ with Alpine as a base image, and install the JDK of our choice:

```
FROM alpine:edge
MAINTAINER baeldung.com
RUN apk add --no-cache openjdk8
```

-   **FROM**: The keyword _FROM_ tells _Docker_ to use a given image with its tag as build-base. If this image isn't in the local library, an online-search on _[DockerHub](https://hub.docker.com/explore/)_, or on any other configured remote-registry, is performed.
-   **MAINTAINER**: A _MAINTAINER_ is usually an email address, identifying the author of an image.
-   **RUN**: With the _RUN_ command, we're executing a shell command-line within the target system. Here we're utilizing _Alpine Linux's_ package manager, _apk,_ to install the _Java 8 OpenJDK._

To finally build the image and store it in the local library, we have to run:

```
docker build --tag=alpine-java:base --rm=true .
```

**NOTICE:** The _–tag_ option will give the image its name, and _–rm=true_ will remove intermediate images after it's been built successfully. The last character in this shell command is a dot, acting as a build-directory argument.

Now we can use the created image instead of _openjdk:8-jdk-alpine_.

## 5\. Buildpacks Support in Spring Boot 2.3[](https://www.baeldung.com/dockerizing-spring-boot-application#buildpack)

**Spring Boot 2.3 added support for** [**buildpacks**](https://buildpacks.io/). Put simply, instead of creating our own Dockerfile and building it using something like _docker build_, all we have to do is issue the following command:

```
$ mvn spring-boot:build-image
```

Similarly, in Gradle:

```
$ ./gradlew bootBuildImage
```

**For this to work, we need to have Docker installed and running.**

The main motivation behind buildpacks is to create the same deployment experience that some well-known cloud services, such as Heroku or Cloud Foundry, have been providing for a while. We just run the _build-image_ goal, and then the platform itself takes care of building and deploying the artifact.

Moreover, it can help us change the way we're [building Docker images more effectively](https://www.baeldung.com/spring-boot-docker-images). Instead of applying the same change to lots of Dockerfiles in different projects, all we have to do is change or tune the buildpacks image builder.

**In addition to ease of use and better overall developer experience, it can also be more efficient.** For instance, the buildpacks approach will create a layered Docker image and uses the exploded version of the Jar file.

Let's look at what happens after we run the above command.

When we list the available docker images:

```
docker image ls -a
```

We see a line for the image we just created:

```
docker-message-server 1.0.0 b535b0cc0079
```

Here, the image name and version match the name and version we defined in the Maven or Gradle configuration file. The hash code is the short version of the image's hash.

Then to start our container, we can simply run:

```
docker run -it -p9099:8888 docker-message-server:1.0.0
```

As with our built image, we need to map the port to make our Spring Boot application accessible from outside Docker.

## **6\. Conclusion**[](https://www.baeldung.com/dockerizing-spring-boot-application#Conclusion)

In this article, we learned how to build custom _Docker_ images, run a _Spring Boot Application_ as a _Docker_ container, and create containers with _docker-compose_.

For further reading about the build files, we refer to the official _[Dockerfile reference](https://docs.docker.com/engine/reference/builder/)_ and the _[docker-compose.yml reference](https://docs.docker.com/compose/compose-file/)_.

As usual, the source codes for this article can be found _[on Github](https://github.com/eugenp/tutorials/tree/master/spring-cloud-modules/spring-cloud-docker)_.
